In [41]:
# Data Manipulation and Handling
import polars as pl
import pandas as pd
import numpy as np
import psycopg2

# DB Credentials
from dotenv import load_dotenv
import os

# Machine Learning Libraries
import torch
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Handling Imbalanced Data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

# Gradient Boosting Libraries
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Model Lifecycle Management
import mlflow
import mlflow.sklearn

# Distributed Computing
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier as SparkRFClassifier

# Model Interpretability
import shap

# Hyperparameter Optimization
import optuna

# Automated Feature Engineering
import featuretools as ft

%load_ext dotenv
%dotenv
BASE_DIR = '../SQL/'


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


## Step 1: Load the Data

In [42]:
equity_data = pd.read_csv(r'../../data/equity_value_data.csv', parse_dates=['timestamp'])
features_data =  pd.read_csv(r'../../data/features_data.csv')

In [43]:
equity_data

,timestamp,close_equity,user_id
0,2016-11-16 00:00:00+00:00,48.16,bcef4fa9b0bdf22bcf7deae708decf03
1,2016-11-17 00:00:00+00:00,48.16,bcef4fa9b0bdf22bcf7deae708decf03
2,2016-11-18 00:00:00+00:00,48.16,bcef4fa9b0bdf22bcf7deae708decf03
3,2016-11-21 00:00:00+00:00,48.16,bcef4fa9b0bdf22bcf7deae708decf03
4,2016-11-22 00:00:00+00:00,48.16,bcef4fa9b0bdf22bcf7deae708decf03
...,...,...,...
1119153,2017-08-14 00:00:00+00:00,2270.71,98580360c4fb5b0ec511cd87f0d429ed
1119154,2017-08-15 00:00:00+00:00,2275.02,98580360c4fb5b0ec511cd87f0d429ed
1119155,2017-08-16 00:00:00+00:00,2282.03,98580360c4fb5b0ec511cd87f0d429ed
1119156,2017-08-17 00:00:00+00:00,2237.45,98580360c4fb5b0ec511cd87f0d429ed


In [44]:
features_data

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id
0,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,33.129417,stock,40.0,med_time_horizon,895044c23edc821881e87da749c01034
1,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,16.573517,stock,200.0,short_time_horizon,458b1d95441ced242949deefe8e4b638
2,med_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,10.008367,stock,25.0,long_time_horizon,c7936f653d293479e034865db9bb932f
3,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,1.031633,stock,100.0,short_time_horizon,b255d4bd6c9ba194d3a350b3e76c6393
4,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,8.187250,stock,20.0,long_time_horizon,4a168225e89375b8de605cbc0977ae91
...,...,...,...,...,...,...,...,...,...
5579,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,8.339283,stock,300.0,long_time_horizon,03880c726d8a4e5db006afe4119ad974
5580,med_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,7.241383,stock,100.0,short_time_horizon,ae8315109657f44852b24c6bca4decd6
5581,med_risk_tolerance,no_investment_exp,very_important_liq_need,both,22.967167,stock,50.0,short_time_horizon,f29c174989f9737058fe808fcf264135
5582,med_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,10.338417,stock,100.0,long_time_horizon,24843497d1de88b2e7233f694436cb3a


In [45]:
features_data.dtypes
#features_data['time_spent'].dtypes

dtype('float64')

In [46]:
features_data['time_spent'] = pd.to_datetime(features_data['time_spent'])
features_data.dtypes

risk_tolerance                          object
investment_experience                   object
liquidity_needs                         object
platform                                object
time_spent                      datetime64[ns]
instrument_type_first_traded            object
first_deposit_amount                   float64
time_horizon                            object
user_id                                 object
dtype: object

In [47]:
equity_data['timestamp'] = pd.to_datetime(equity_data['timestamp'])
equity_data.dtypes

timestamp       datetime64[ns, UTC]
close_equity                float64
user_id                      object
dtype: object

In [48]:
equity_data['timestamp'] = equity_data['timestamp'].dt.tz_localize(None)
equity_data.dtypes

timestamp       datetime64[ns]
close_equity           float64
user_id                 object
dtype: object

## Step 2: Generate the Complete Calendar

In [49]:
# Get the min and max dates from the equity data
min_date = equity_data['timestamp'].min()
max_date = equity_data['timestamp'].max()

# Create a date range of all calendar dates
all_dates = pd.date_range(start=min_date, end=max_date, freq='D')
all_dates

DatetimeIndex(['2016-08-16', '2016-08-17', '2016-08-18', '2016-08-19',
               '2016-08-20', '2016-08-21', '2016-08-22', '2016-08-23',
               '2016-08-24', '2016-08-25',
               ...
               '2017-08-09', '2017-08-10', '2017-08-11', '2017-08-12',
               '2017-08-13', '2017-08-14', '2017-08-15', '2017-08-16',
               '2017-08-17', '2017-08-18'],
              dtype='datetime64[ns]', length=368, freq='D')

## Step 3: Identify Market Open and Closed Days

In [50]:
# Determine market open days (dates with any data)
market_open_days = equity_data['timestamp'].dt.date.unique()

# Create a DataFrame for all dates with market status
calendar_df = pd.DataFrame({'date': all_dates})
calendar_df['market_status'] = calendar_df['date'].dt.date.apply(
    lambda x: 'open' if x in market_open_days else 'closed'
)


## Step 4: Prepare User-Date Data

In [51]:
# Get the list of all users
user_ids = equity_data['user_id'].unique()

# Create a MultiIndex with all users and all dates
user_date_index = pd.MultiIndex.from_product(
    [user_ids, all_dates], names=['user_id', 'date']
)

# Create a DataFrame with this index
user_date_df = pd.DataFrame(index=user_date_index).reset_index()

# Merge with the calendar DataFrame to get market status
user_date_df = user_date_df.merge(calendar_df, on='date', how='left')

## Step 5: Merge Equity Data

In [52]:
# print("Data types before conversion:")
# print("user_date_df['date'] dtype:", user_date_df['date'].dtype)
# print("equity_data['date'] dtype:", equity_data['date'].dtype)

# equity_data['date'] = pd.to_datetime(equity_data['date'])
# equity_data.dtypes


In [53]:
# Prepare equity data by setting the index
equity_data['date'] = equity_data['timestamp'].dt.date
equity_data['date'] = pd.to_datetime(equity_data['date'])

equity_data = equity_data[['user_id', 'date', 'close_equity']]

# Merge user-date DataFrame with equity data
user_date_df = user_date_df.merge(
    equity_data, on=['user_id', 'date'], how='left'
)


## Step 6: Determine Equity State

In [54]:
# Determine the equity state
def determine_state(row):
    if pd.notnull(row['close_equity']):
        return 'equity_ge_10'
    elif row['market_status'] == 'open':
        return 'equity_lt_10'
    else:
        return None  # Will fill this later

user_date_df['state'] = user_date_df.apply(determine_state, axis=1)


## Step 7: Carry Forward the Last Known State

In [55]:
# Sort the DataFrame
user_date_df = user_date_df.sort_values(['user_id', 'date'])

# Group by user and fill forward the state
user_date_df['state'] = user_date_df.groupby('user_id')['state'].ffill()

# For any remaining missing states at the beginning, fill with 'equity_ge_10'
user_date_df['state'] = user_date_df['state'].fillna('equity_ge_10')


## Step 8: Identify Consecutive Periods of Equity < $10

In [56]:
# # Ensure the DataFrame is sorted by 'user_id' and 'date'
# user_date_df = user_date_df.sort_values(['user_id', 'date']).reset_index(drop=True)

# # Identify consecutive sequences
# user_date_df['grp'] = (
#     user_date_df.groupby('user_id')['below_10']
#     .apply(lambda x: (x != x.shift()).cumsum())
#     .reset_index(level=0, drop=True)
# )
# # Filter only the periods where equity was below $10
# # below_10_df = user_date_df[user_date_df['below_10']]
# # Option 2
# # # Ensure the DataFrame is sorted
# # user_date_df = user_date_df.sort_values(['user_id', 'date']).reset_index(drop=True)

# # # Identify changes in 'below_10' and compute cumulative sum
# # user_date_df['grp'] = user_date_df.groupby('user_id')['below_10'].transform(
# #     lambda x: (x != x.shift()).cumsum()
# # )
# # Option 3
# # # Ensure the DataFrame is sorted
# # user_date_df = user_date_df.sort_values(['user_id', 'date']).reset_index(drop=True)

# # # Identify consecutive sequences
# # user_date_df['grp'] = (
# #     (user_date_df['below_10'] != user_date_df.groupby('user_id')['below_10'].shift())
# #     .astype(int)
# #     .groupby(user_date_df['user_id'])
# #     .cumsum()
# # )

# # Final Update
# Flag days where equity is less than $10
user_date_df['below_10'] = user_date_df['state'] == 'equity_lt_10'

# Ensure the DataFrame is sorted
user_date_df = user_date_df.sort_values(['user_id', 'date']).reset_index(drop=True)

# Identify consecutive sequences without using apply()
user_date_df['grp'] = (
    (user_date_df['below_10'] != user_date_df.groupby('user_id')['below_10'].shift())
    .fillna(1)  # Handle NaN values resulting from the shift
    .astype(int)
    .groupby(user_date_df['user_id'])
    .cumsum()
)

# Filter only the periods where equity was below $10
below_10_df = user_date_df[user_date_df['below_10']]



## Step 9: Calculate Sequence Lengths

In [57]:
# Calculate the length of each sequence
sequence_lengths = (
    below_10_df.groupby(['user_id', 'grp'])
    .agg(start_date=('date', 'min'), end_date=('date', 'max'), num_days=('date', 'count'))
    .reset_index()
)

# Calculate the number of calendar days in each sequence
sequence_lengths['num_calendar_days'] = (
    sequence_lengths['end_date'] - sequence_lengths['start_date']
).dt.days + 1  # Add 1 to include both start and end dates
# # Calculate the length of each sequence
# sequence_lengths = (
#     user_date_df[user_date_df['below_10']]
#     .groupby(['user_id', 'grp'])
#     .agg(
#         start_date=('date', 'min'),
#         end_date=('date', 'max'),
#         num_days=('date', 'count')
#     )
#     .reset_index()
# )

# # Calculate the number of calendar days
# sequence_lengths['num_calendar_days'] = (
#     sequence_lengths['end_date'] - sequence_lengths['start_date']
# ).dt.days + 1


## Step 10: Identify Users Who Have Churned

In [58]:
# Identify sequences where num_calendar_days >= 28
churned_sequences = sequence_lengths[sequence_lengths['num_calendar_days'] >= 28]

# Get the list of churned users
churned_users = churned_sequences['user_id'].unique()

# # Identify sequences where num_calendar_days >= 28
# churned_sequences = sequence_lengths[sequence_lengths['num_calendar_days'] >= 28]

# # Get the list of churned users
# churned_users = churned_sequences['user_id'].unique()


## Step 11: Calculate the Percentage of Users Who Have Churned

In [59]:
total_users = len(user_ids)
num_churned_users = len(churned_users)
percentage_churned = (num_churned_users / total_users) * 100

print(f"Percentage of users who have churned: {percentage_churned:.2f}%")
# # Ensure the DataFrame is sorted
# user_date_df = user_date_df.sort_values(['user_id', 'date']).reset_index(drop=True)

# # Identify consecutive sequences
# user_date_df['grp'] = (
#     (user_date_df['below_10'] != user_date_df.groupby('user_id')['below_10'].shift())
#     .astype(int)
#     .groupby(user_date_df['user_id'])
#     .cumsum()
# )

Percentage of users who have churned: 45.38%
